# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [6]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core.model import Model, InferenceConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, loguniform
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.webservice import AciWebservice
import sklearn
import os
import joblib
import pandas as pd
import json

## Dataset

Data accessed and cleaned in the train.py script

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

## Start compute cluster

In [3]:
cpu_cluster_name = "ML-Cluster2"
compute_config = AmlCompute.provisioning_configuration("Standard_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [12]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.05, evaluation_interval=5, delay_evaluation=10)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": loguniform(-10, 10),
    "--max_iter": choice(50,100,200,400)
})

param_sampling = BayesianParameterSampling({
    "--C": uniform(0.01, 1000),
    "--max_iter": choice(50,100,200,400)
})


#TODO: Create your estimator and hyperdrive config
estimator = SKLearn("./", compute_target=cpu_cluster, entry_script="./source_dir/train.py")

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling, 
                                    estimator=estimator, 
                                   # policy=early_termination_policy,
                                    primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  
                                    max_total_runs=40)

We recommend using 20 max concurrent runs or fewer when using Bayesian sampling since a higher number might not provide the best result.


In [13]:
#TODO: Submit your experiment
run = experiment.submit(hyperdrive_run_config, show_output=True)
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
best_run = run.get_best_run_by_primary_metric()
best_run_metric = best_run.get_metrics()

print(best_run)
print(best_run_metric)

best_run.get_file_names()

if "outputs" not in os.listdir():
    os.mkdir("./outputs")


best_model=best_run.register_model(
    model_name='hdr', 
    model_path='outputs/model.joblib',
    tags=best_run_metric, 
    model_framework=Model.Framework.SCIKITLEARN, 
    model_framework_version=sklearn.__version__)

# Save the best model
#best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/')

Run(Experiment: hyperdrive,
Id: HD_580a0fc9-3583-46ce-823d-e2e4a4879961_3,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 785.8235024567775, 'Max iterations:': 200, 'Accuracy': 0.6702230843840931}


In [15]:
metrics = pd.DataFrame.from_dict(best_run_metric, orient='index', columns=['metric value'])
metrics

,metric value
Regularization Strength:,785.823502
Max iterations:,200.000000
Accuracy,0.670223


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
# Define inference config

env = best_run.get_environment()
inference_config = InferenceConfig(
    environment= env,
    source_directory = "./source_dir",
    entry_script="score.py"
)

In [17]:
# Define a deployment configuration on the Webservice

deploy_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True, enable_app_insights=True
)

In [18]:
# Deploy the model

service = Model.deploy(
    workspace = ws,
    name = "sinter-scrap-model2",
    models = [best_model],
    inference_config = inference_config,
    deployment_config = deploy_config,
    overwrite = True
)

service.wait_for_deployment(show_output=True)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-03 16:58:49+00:00 Creating Container Registry if not exists..
2021-11-03 17:08:50+00:00 Registering the environment.
2021-11-03 17:08:52+00:00 Use the existing image.
2021-11-03 17:08:52+00:00 Generating deployment configuration.
2021-11-03 17:08:54+00:00 Submitting deployment to compute..
2021-11-03 17:08:59+00:00 Checking the status of deployment sinter-scrap-model2..
2021-11-03 17:14:01+00:00 Checking the status of inference endpoint sinter-scrap-model2.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-11-03T17:13:32,794370700+00:00 - iot-server/run 
2021-11-03T17:13:32,818324300+00:00 - rsyslog/run 
2021-11-03T17:13:32,828395700+00:00 - nginx/run 
2021-11-03T17:13:32,848401500+00:00 - gunicorn/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [19]:
raw_data = pd.read_csv('./data/balanced.csv')
raw_data = raw_data.dropna()
raw_data = raw_data.drop("WO-MRR NUM", axis=1)
raw_data = raw_data.drop("Has MRR", axis=1)
raw_data = pd.get_dummies(raw_data, columns=['PROD_CDE','PREP ASSOCIATE', 'FURN'])

# Use the first 10 rows of the actual cleaned data as a test data
test_data = raw_data[1000:1010]

# Drop the column that will be predicted
test_data = test_data.drop("HAS SCRAP AT SINTER", axis=1)

# Convert pandas DataFrame into the dictionary
test_dict = ({'data':test_data.to_dict(orient='records')})

# Encode data into the JSON object
test_sample = json.dumps(test_dict)


# Pass the JSON object into our deployed models endpoint
# and print the predicted results
print(service.run(test_sample))

[1, 0, 0, 0, 0, 0, 1, 0, 1, 0]


TODO: In the cell below, print the logs of the web service and delete the service

In [20]:
print(service.get_logs())

2021-11-03T17:13:32,794370700+00:00 - iot-server/run 
2021-11-03T17:13:32,818324300+00:00 - rsyslog/run 
2021-11-03T17:13:32,828395700+00:00 - nginx/run 
2021-11-03T17:13:32,848401500+00:00 - gunicorn/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-11-03T17:13:33,521528300+00:00 - iot-server/finish 1 0
2021-11-03T17:13:33,523398200+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (67)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 99
Initialized PySpark session.
Initializing logger
Starting up app insights client
Starting up request id generator
Starting up app insight hooks
Invoking user's init function
Users's init has completed successfully
Skipping middleware: dbg_model_info as it's not enabled.
Scoring timeout is found from os.environ: 60000 ms
Swagger file not present
404
127.0.0.1 - - [03/Nov/2021:17:14:01 +0000] "GET /swagger.json HTTP/1.0" 404 1

In [42]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

